In [ ]:
import os 
from time import sleep
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup
import warnings 
import urllib
warnings.filterwarnings('ignore')


#블로그 이미지 추출 함수
def get_img_list(html):
    soup = BeautifulSoup(html, 'html.parser')
    imgs = soup.select('.se-image-resource')
    img_list = []
    for img in imgs:
        try:
            img_list.append(img['data-lazy-src'])  # 큰사진의 경로
            #print(img['data-lazy-src'])
            try:
                img_list.append(img['src'])  # 큰사진의 경로
                #print( img['src'] )
            except:
                continue
        except:
            continue
    return img_list 

# 저장경로 생성 함수
def mkdir(title):
    save_dest = 'C:/Users/ecopl/image_blog/'  # 저장을 위한 베이스 경로
    if os.path.isdir(save_dest + title):  # 베이스경로에 제목으로 하위 폴더 생성
        pass
    else:
        os.makedirs(save_dest + title)
    return save_dest + title

# 이미지 저장 함수
def save_img(img_list, save_dest): 
    #print("img_list길이:",len(img_list))
    try:
        for i in range(len(img_list)-10, len(img_list)): 
            ext_idx = (img_list[i].find('?type')) #확장자 찾기 위해 type 문자열을 먼저 찾고 
            ext = (img_list[i][ext_idx-3:ext_idx]) #위에서 찾은 인덱스에서 -3하여 해당 파일의 확장자 확인 
            try:
                urllib.request.urlretrieve(img_list[i], save_dest+'/'+str(i+1)+'.'+ext)
                #print("링크인식",i)
            except:
                continue
            sleep(0.1)
    except:
        print("사진없는 블로그")
    
    

chromedriver_path = "chromedriver"
driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path))  # chromedriver 열기
###

place_name_list=[] #장소 이름
#place_review_list=[]
place_alt_review_list=[] #장소의 해당 블로그 리뷰가 alt인지아닌지
review_date_list=[]
content_list=[]



def main():
    global driver
    
    driver.implicitly_wait(4)  # 렌더링 될때까지 기다린다 4초
    driver.get('https://map.naver.com/')  # 주소 가져오기
    
    #검색 목록
    place_infos=['강남 맛집']
    
    for i, place in enumerate(place_infos):
        # delay
        if i % 4 == 0 and i != 0:
            sleep(5)
        print("#####", place, " 검색 결과 크롤링 시작.")
        search(place) #search사용자정의함수 호출

    driver.quit()
    print("크롤링 끝")
    
def search(place):
    global driver
    
    search_area = driver.find_element_by_xpath('/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-input-box/div/div[1]/div/input')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element_by_xpath('/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-input-box/div/div[1]/div/input').send_keys(Keys.ENTER)  # Enter로 검색
    sleep(1)
    page=1
    #총 6페이지 돌아가며 가게 크롤링
    for i in range(0,3):
        driver.switch_to.frame("searchIframe")
        place_list_area = driver.find_element_by_xpath('//*[@id="_pcmap_list_scroll_container"]')  # 장소목록 스크롤 창 찾기
        for i in range(6):
            #last_height = driver.execute_script("return document.body.scrollHeight")
            # 끝까지 스크롤 내리기
            driver.execute_script("arguments[0].scrollBy(0, 10000)", place_list_area)
            sleep(1)



        # 위에서부터 순서대로 가게 상세정보 iframe창 열고 크롤링
        idx=3
        count=0
        
        #가게 5개 크롤링할때마다 창 새로띄워주기
        while True:
            if count >=5:
                count=0
                driver.quit()
                chromedriver_path = "chromedriver"
                driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path))  # chromedriver 열기
                driver.implicitly_wait(2)  # 렌더링 될때까지 기다린다 4초
                driver.get('https://map.naver.com/')  # 주소 가져오기
                search_area = driver.find_element_by_xpath('/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-input-box/div/div[1]/div/input')  # 검색 창
                search_area.send_keys(place)  # 검색어 입력
                driver.find_element_by_xpath('/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-input-box/div/div[1]/div/input').send_keys(Keys.ENTER)  # Enter로 검색
                sleep(1)
                for i in range(page-1):
                    driver.switch_to.frame("searchIframe")
                    driver.find_element_by_xpath('//*[@id="app-root"]/div/div[2]/div[2]/a[7]').click()
                    sleep(1)
                    driver.switch_to.default_content() #처음 frame으로 넘어오기
                driver.switch_to.frame("searchIframe")
                place_list_area = driver.find_element_by_xpath('//*[@id="_pcmap_list_scroll_container"]')  # 장소목록 스크롤 창 찾기
                for i in range(5):
                    #last_height = driver.execute_script("return document.body.scrollHeight")
                    # 끝까지 스크롤 내리기
                    driver.execute_script("arguments[0].scrollBy(0, 10000)", place_list_area)
                    sleep(1)
                
            detail_page_xpath ='//*[@id="_pcmap_list_scroll_container"]/ul/li['+ str(idx) +']/div[1]/a[1]'

            try:
                driver.find_element_by_xpath(detail_page_xpath).click()
            except:
                try:
                    driver.find_element_by_xpath(detail_page_xpath).send_keys(Keys.ENTER)
                except:
                    break
            driver.switch_to.default_content() #처음 frame으로 넘어오기
            #가게 상세정보 iframe창으로 변경
            sleep(1)
            driver.switch_to.frame("entryIframe")
            sleep(1)
            #리뷰탭으로 변경
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            infotab= soup.select('.flicking-camera > a')
            #print('infotab길이:',len(infotab))
            if (len(infotab) == 4) :
                #driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div[2]/div[3]/div/div/div/div/a[3]').click()
                driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div/div[5]/div/div/div/div/a[3]').click()
                try:
                    driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div/div[6]/div[1]/div/a[2]').click()
                except:
                    print('review click fail')
            elif (len(infotab) == 5) :
                #driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div[2]/div[3]/div/div/div/div/a[4]').click()
                driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div/div[5]/div/div/div/div/a[4]').click()
                try:
                    driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div/div[6]/div[1]/div/a[2]').click()
                except:
                    print('review click fail')    
            elif (len(infotab) == 6) :
                #driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div[2]/div[3]/div/div/div/div/a[5]').click()
                driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div/div[5]/div/div/div/div/a[5]').click()
                try:
                    driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div/div[6]/div[1]/div/a[2]').click()
                except:
                    print('review click fail')
            elif (len(infotab) == 7) :
                #driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div[2]/div[3]/div/div/div/div/a[5]').click()
                driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div/div[5]/div/div/div/div/a[5]').click()
                try:
                    driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div/div[6]/div[1]/div/a[2]').click()
                except:
                    print('review click fail')
            sleep(1)
            driver.execute_script("window.scrollTo(0, 1000)")
            sleep(1)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            place_name=soup.select('._1_hm2 > span')
            print('가게이름:',place_name[0].text)
            
            #더보기 30회만큼 누르기
            print("블로그리스트 탐색중...")
            for i in range(30):
                try:
                    driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[2]/div[2]').click()
                    sleep(0.1)
                except:
                    continue
            sleep(2)
            print("탐색끝")
            #블로그 클릭, 접속
            #place_review_count=0
            #place_alt_review_count=0
            for i in range(1,300):
                
                try:
                    #driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[2]/div[1]/ul/li['+str(i)+']/a/div[1]').click()
                    driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[2]/div[1]/ul/li['+str(i)+']/a[1]').send_keys(Keys.ENTER)
                    sleep(1)
                    #블로그탭으로 변경
                    driver.switch_to.window(driver.window_handles[-1])
                    #블로그 리뷰 내 메인프레임으로 프레임 이동
                    try:
                        driver.switch_to.frame("mainFrame")
                    except:
                        driver.close() # 블로그탭 닫기
                        sleep(1)
                        driver.switch_to.window(driver.window_handles[0]) #상세화면으로 전환
                        sleep(1)
                        driver.switch_to.frame("entryIframe")
                        continue
                    sleep(1)
                    
                except:
                    print('블로그링크접속실패')
                    break
                
                try:
                    text=driver.find_element_by_css_selector("div.se-main-container").text
                except: #위방식으로 크롤링 불가한 블로그 형식
                    driver.close() #블로그탭닫기
                    sleep(0.5)
                    driver.switch_to.window(driver.window_handles[0]) #상세화면으로 전환
                    sleep(0.5)
                    driver.switch_to.frame("entryIframe")
                    sleep(0.5)
                    continue
                #place_review_count += 1
                #print(text)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                review_date=soup.select('.blog2_container > span')
                print(review_date[1].text)
                
                
                #블로그 이미지 크롤링
                img_list=get_img_list(html)
                save_dest=mkdir(place_name[0].text + str(i))
                img_cnt = save_img(img_list, save_dest)
                
                
                if ('원고료' in text) | ('소정의' in text) | ('고료' in text) | ('업체로부터' in text) | ('쿠팡파트너스' in text):
                    print('광고리뷰확인')
                    place_alt_review_list.append(1)
                else:
                    print('정상리뷰')
                    place_alt_review_list.append(0)
                place_name_list.append(place_name[0].text+str(i)) #장소이름 스택
                review_date_list.append(review_date[1].text) #리뷰날짜 스택
                content_list.append(text) #블로그 글내용 스택
                
                sleep(1)
                try:
                    driver.close() #블로그탭닫기
                except:
                    driver.close() # 블로그탭 닫기
                sleep(1)
                driver.switch_to.window(driver.window_handles[0]) #상세화면으로 전환
                sleep(1)
                driver.switch_to.frame("entryIframe")
                sleep(1)
            #place_name_list.append()
            #place_review_list.append(place_review_count)
            #place_alt_review_list.append(place_alt_review_count)
            #print(place_review_list)
            #print(review_date_list)
            idx += 1
            driver.switch_to.default_content() #처음 frame으로 넘어오기
            driver.switch_to.frame("searchIframe") #search frame으로 넘어가기
            count+=1
        print("다음 장소목록 페이지로 넘어갑니다.")
        try:
            driver.find_element_by_xpath('//*[@id="app-root"]/div/div[2]/div[2]/a[7]').click()
            sleep(2)
        except:
            try:
                driver.find_element_by_xpath('//*[@id="app-root"]/div/div[2]/div[2]/a[7]').click()
                sleep(2)
            except:
                print("가게목록 마지막 페이지입니다.")
                break
        page +=1
        driver.switch_to.default_content() #처음 frame으로 넘어오기
                

            
            
            

if __name__ == "__main__":
    main()
    data = {"place_name": place_name_list, "date": review_date_list, "content": content_list, "altificial_simple": place_alt_review_list}
    df = pd.DataFrame(data, columns=["place_name","date","content","altificial_simple"])
    df.to_csv("22summeer개별연구크롤링220628.csv", index=False, encoding='utf-8-sig')

##### 강남 맛집  검색 결과 크롤링 시작.
가게이름: 바비레드 강남본점
블로그리스트 탐색중...
탐색끝
16시간 전
광고리뷰확인
2022. 6. 2. 17:42
광고리뷰확인
2022. 5. 27. 22:54
광고리뷰확인
2022. 5. 27. 23:52
광고리뷰확인
2022. 6. 22. 12:47
정상리뷰
2022. 6. 2. 13:52
광고리뷰확인
2022. 6. 9. 15:34
정상리뷰
2022. 6. 18. 8:00
정상리뷰
2022. 5. 30. 23:50
광고리뷰확인
2022. 4. 11. 14:36
정상리뷰
2022. 5. 31. 1:19
정상리뷰
2022. 5. 26. 9:38
정상리뷰
2022. 5. 27. 18:25
광고리뷰확인
2022. 4. 26. 14:51
정상리뷰
19시간 전
정상리뷰
2022. 4. 26. 12:43
광고리뷰확인
2022. 6. 1. 12:39
정상리뷰
2022. 2. 17. 22:11
정상리뷰
2022. 3. 28. 11:05
광고리뷰확인
2022. 1. 18. 22:45
정상리뷰
2021. 10. 13. 23:36
정상리뷰
2021. 12. 8. 21:07
정상리뷰
2021. 4. 23. 14:57
정상리뷰
2021. 12. 23. 0:00
정상리뷰
2021. 9. 15. 23:21
정상리뷰
2021. 11. 7. 23:44
정상리뷰
2021. 11. 26. 22:06
정상리뷰
2022. 6. 18. 23:23
정상리뷰
9시간 전
정상리뷰
2022. 6. 26. 0:40
정상리뷰
2022. 6. 13. 22:43
정상리뷰
2022. 6. 21. 11:40
정상리뷰
2022. 6. 10. 23:20
정상리뷰
2022. 6. 26. 17:03
정상리뷰
2022. 6. 19. 22:13
정상리뷰
2022. 6. 9. 7:30
정상리뷰
2022. 5. 31. 10:27
정상리뷰
2022. 6. 26. 22:16
정상리뷰
2022. 6. 7. 13:44
정상리뷰
2022. 6. 9. 20:46
정상리뷰
2022. 6.

2022. 1. 28. 6:27
정상리뷰
2022. 5. 30. 9:44
정상리뷰
2020. 6. 24. 0:39
정상리뷰
2022. 4. 8. 6:30
정상리뷰
2020. 12. 19. 12:02
정상리뷰
2020. 11. 25. 14:27
정상리뷰
2022. 4. 15. 10:24
정상리뷰
2020. 10. 12. 20:42
정상리뷰
2021. 11. 10. 9:14
정상리뷰
2021. 6. 22. 17:23
정상리뷰
2021. 12. 20. 8:15
정상리뷰
2021. 4. 26. 15:30
정상리뷰
2022. 1. 7. 15:00
사진없는 블로그
정상리뷰
2021. 12. 10. 10:37
정상리뷰
2021. 4. 18. 9:49
정상리뷰
2022. 1. 7. 7:51
정상리뷰
2021. 11. 11. 15:00
정상리뷰
2022. 1. 15. 21:44
정상리뷰
2021. 8. 7. 14:40
정상리뷰
2020. 8. 16. 7:20
정상리뷰
2021. 9. 21. 21:13
정상리뷰
2020. 6. 4. 20:44
정상리뷰
2021. 11. 19. 15:01
정상리뷰
2021. 8. 20. 0:44
정상리뷰
2021. 5. 25. 0:27
정상리뷰
2020. 7. 30. 23:14
정상리뷰
2021. 8. 20. 20:31
정상리뷰
2020. 11. 27. 17:27
정상리뷰
2021. 3. 24. 16:36
정상리뷰
2021. 5. 6. 15:00
정상리뷰
2020. 7. 6. 23:03
정상리뷰
2021. 5. 15. 23:47
정상리뷰
2020. 5. 27. 15:15
정상리뷰
2021. 3. 21. 18:46
정상리뷰
2020. 6. 3. 13:57
정상리뷰
2020. 8. 14. 10:23
정상리뷰
2021. 1. 23. 10:15
정상리뷰
2020. 10. 3. 17:30
정상리뷰
블로그링크접속실패
가게이름: 삼원가든
블로그리스트 탐색중...
탐색끝
2022. 6. 27. 8:20
정상리뷰
2022. 6. 23. 16:22
정상리뷰
202

정상리뷰
2022. 6. 9. 10:05
정상리뷰
2022. 5. 29. 21:10
정상리뷰
2022. 1. 5. 14:34
광고리뷰확인
2022. 2. 14. 20:57
정상리뷰
2021. 12. 31. 17:18
정상리뷰
2022. 2. 22. 12:50
정상리뷰
2022. 1. 10. 15:51
정상리뷰
2022. 2. 14. 21:00
정상리뷰
2022. 1. 13. 10:06
정상리뷰
2022. 2. 17. 20:01
정상리뷰
2022. 2. 1. 21:17
정상리뷰
2022. 1. 3. 13:16
정상리뷰
블로그링크접속실패
가게이름: 루카511
블로그리스트 탐색중...
탐색끝
2021. 11. 9. 14:58
정상리뷰
2022. 5. 30. 22:49
정상리뷰
2022. 5. 25. 16:33
정상리뷰
2022. 5. 17. 13:45
정상리뷰
2022. 6. 20. 21:27
정상리뷰
2022. 6. 26. 10:37
정상리뷰
2022. 6. 26. 20:11
정상리뷰
2022. 6. 17. 14:35
정상리뷰
2021. 9. 2. 12:49
정상리뷰
2022. 6. 20. 8:03
정상리뷰
2022. 6. 26. 19:43
정상리뷰
2022. 5. 10. 21:52
정상리뷰
2021. 6. 19. 12:32
정상리뷰
2021. 6. 1. 21:26
정상리뷰
2022. 6. 14. 0:18
정상리뷰
2022. 5. 11. 20:00
정상리뷰
2022. 4. 1. 23:12
정상리뷰
2022. 4. 18. 17:15
정상리뷰
2021. 8. 26. 22:29
정상리뷰
2022. 5. 8. 9:29
정상리뷰
2021. 11. 9. 14:58
정상리뷰
2022. 4. 2. 9:06
정상리뷰
2022. 2. 2. 13:11
정상리뷰
2022. 1. 19. 22:39
정상리뷰
2021. 6. 17. 10:50
정상리뷰
2022. 3. 10. 0:30
정상리뷰
2021. 12. 31. 21:56
정상리뷰
2022. 2. 25. 1:29
정상리뷰
2021. 1

정상리뷰
2022. 6. 13. 6:48
정상리뷰
2022. 6. 24. 1:46
정상리뷰
2021. 12. 9. 22:54
정상리뷰
2021. 11. 17. 12:07
정상리뷰
2021. 8. 21. 21:30
정상리뷰
2020. 7. 14. 16:11
정상리뷰
9시간 전
정상리뷰
2022. 2. 4. 21:02
정상리뷰
2021. 9. 9. 15:50
정상리뷰
2021. 5. 13. 16:00
정상리뷰
2022. 6. 3. 16:41
정상리뷰
2020. 9. 18. 15:56
정상리뷰
2021. 2. 17. 7:13
정상리뷰
2022. 6. 3. 11:55
광고리뷰확인
2021. 9. 2. 11:12
사진없는 블로그
정상리뷰
2019. 2. 18. 16:20
정상리뷰
2020. 5. 19. 22:32
정상리뷰
2022. 2. 6. 13:11
정상리뷰
2022. 5. 28. 20:35
정상리뷰
2019. 8. 15. 15:26
정상리뷰
2019. 1. 28. 22:49
정상리뷰
2022. 6. 1. 20:35
정상리뷰
2022. 5. 24. 9:00
정상리뷰
2022. 4. 14. 10:00
정상리뷰
2022. 5. 22. 23:03
정상리뷰
2022. 6. 9. 9:46
정상리뷰
2022. 5. 9. 17:59
정상리뷰
2022. 6. 5. 21:26
정상리뷰
2022. 3. 29. 23:06
정상리뷰
2022. 4. 21. 9:00
광고리뷰확인
2022. 1. 30. 0:58
정상리뷰
2021. 11. 10. 7:59
정상리뷰
2022. 1. 7. 18:25
정상리뷰
2022. 1. 25. 15:12
정상리뷰
2022. 3. 15. 8:55
정상리뷰
2022. 1. 31. 7:20
정상리뷰
2022. 2. 22. 14:53
정상리뷰
2021. 9. 9. 14:00
정상리뷰
2022. 1. 30. 23:50
정상리뷰
2021. 9. 17. 21:54
정상리뷰
2022. 2. 10. 12:49
정상리뷰
2022. 3. 9. 13:30
정상리뷰
2021. 11

광고리뷰확인
2022. 6. 1. 23:08
정상리뷰
2022. 6. 10. 0:00
정상리뷰
2022. 3. 29. 14:00
정상리뷰
2022. 2. 21. 22:49
정상리뷰
2022. 4. 26. 15:49
정상리뷰
2022. 5. 16. 19:25
정상리뷰
2022. 5. 25. 16:35
정상리뷰
2022. 5. 22. 23:40
정상리뷰
2022. 1. 3. 21:33
정상리뷰
2022. 2. 14. 21:52
정상리뷰
2022. 5. 25. 10:28
사진없는 블로그
정상리뷰
2022. 2. 18. 17:23
광고리뷰확인
2022. 5. 18. 9:00
정상리뷰
2022. 1. 3. 13:55
정상리뷰
2022. 1. 20. 10:39
정상리뷰
2022. 6. 2. 6:48
광고리뷰확인
2022. 1. 4. 20:30
광고리뷰확인
2021. 12. 31. 15:11
정상리뷰
2022. 2. 24. 12:08
정상리뷰
2022. 3. 4. 14:13
정상리뷰
2022. 3. 15. 14:00
정상리뷰
2022. 1. 5. 14:34
광고리뷰확인
2022. 2. 14. 20:57
정상리뷰
2021. 12. 31. 17:18
정상리뷰
2022. 2. 22. 12:50
정상리뷰
2022. 1. 10. 15:51
정상리뷰
2022. 2. 14. 21:00
정상리뷰
2022. 1. 13. 10:06
정상리뷰
2022. 2. 17. 20:01
정상리뷰
2022. 2. 1. 21:17
정상리뷰
2022. 1. 3. 13:16
정상리뷰
2021. 12. 29. 10:51
정상리뷰
2022. 3. 10. 9:26
정상리뷰
2022. 2. 20. 20:00
정상리뷰
2022. 3. 21. 15:03
정상리뷰
2022. 2. 19. 7:10
정상리뷰
2022. 2. 21. 9:00
정상리뷰
2022. 2. 3. 10:41
정상리뷰
2022. 2. 22. 3:23
광고리뷰확인
2022. 2. 27. 0:35
정상리뷰
2022. 4. 22. 22:03
정상리뷰
블로그링크

...

정상리뷰
2022. 6. 24. 10:59
정상리뷰
2022. 6. 17. 23:58
정상리뷰
2022. 6. 9. 16:54
정상리뷰
2022. 6. 9. 7:07
정상리뷰
2022. 6. 20. 0:26
사진없는 블로그
정상리뷰
2022. 6. 4. 19:44
정상리뷰
2022. 6. 6. 15:24
정상리뷰
2022. 6. 1. 19:46
정상리뷰
2022. 6. 14. 14:57
정상리뷰
2022. 6. 7. 8:31
정상리뷰
블로그링크접속실패
가게이름: 도산맘마미아
블로그리스트 탐색중...
탐색끝
2022. 3. 23. 21:00
정상리뷰
2022. 4. 29. 22:54
정상리뷰
2022. 5. 20. 0:01
정상리뷰
2022. 2. 18. 10:00
정상리뷰
2022. 6. 23. 0:13
정상리뷰
2022. 4. 13. 16:22
정상리뷰
2022. 5. 1. 21:04
정상리뷰
2022. 4. 29. 17:19
정상리뷰
2022. 6. 27. 0:10
정상리뷰
2022. 6. 23. 20:03
정상리뷰
2022. 3. 23. 21:00
정상리뷰
2022. 4. 4. 22:08
정상리뷰
2022. 6. 27. 12:33
정상리뷰
2022. 4. 13. 8:46
정상리뷰
2022. 3. 30. 10:30
정상리뷰
2022. 6. 24. 1:03
정상리뷰
2022. 3. 23. 16:22
정상리뷰
2022. 3. 7. 8:30
정상리뷰
2022. 4. 2. 23:18
정상리뷰
2022. 3. 12. 11:49
정상리뷰
2022. 1. 27. 9:10
정상리뷰
2022. 2. 18. 11:52
정상리뷰
2022. 3. 7. 13:35
정상리뷰
2022. 6. 23. 15:13
정상리뷰
2022. 2. 5. 16:10
정상리뷰
2022. 6. 27. 0:50
정상리뷰
2022. 6. 9. 20:05
정상리뷰
2022. 6. 19. 21:50
정상리뷰
2021. 11. 30. 9:00
정상리뷰
2022. 6. 24. 16:06
정상리뷰
2021. 2. 

2021. 6. 27. 16:02
정상리뷰
2021. 5. 10. 0:33
정상리뷰
2021. 6. 6. 11:57
정상리뷰
2022. 6. 16. 14:47
정상리뷰
2021. 7. 5. 1:32
정상리뷰
2019. 11. 3. 15:15
정상리뷰
2020. 9. 4. 12:34
정상리뷰
2020. 11. 25. 22:18
정상리뷰
2020. 9. 28. 8:45
정상리뷰
2022. 5. 17. 23:07
정상리뷰
2019. 2. 6. 23:19
정상리뷰
2018. 12. 16. 8:05
정상리뷰
2022. 6. 14. 21:26
정상리뷰
2022. 6. 6. 12:27
정상리뷰
2022. 5. 3. 20:40
정상리뷰
2022. 5. 11. 19:56
정상리뷰
2022. 5. 23. 18:30
정상리뷰
2022. 4. 22. 22:07
정상리뷰
2022. 6. 14. 11:33
정상리뷰
2022. 6. 2. 14:00
정상리뷰
2022. 5. 28. 1:33
정상리뷰
2022. 5. 6. 23:51
정상리뷰
2022. 5. 3. 12:00
정상리뷰
2022. 4. 7. 4:51
정상리뷰
2022. 5. 1. 11:41
정상리뷰
2022. 5. 13. 0:12
정상리뷰
2022. 5. 17. 20:45
정상리뷰
2022. 5. 23. 17:39
정상리뷰
2022. 3. 20. 0:10
정상리뷰
2022. 4. 26. 2:48
정상리뷰
2022. 3. 4. 7:02
정상리뷰
2022. 6. 8. 12:58
정상리뷰
2022. 3. 13. 12:43
정상리뷰
2022. 5. 11. 18:53
정상리뷰
2022. 5. 27. 22:57
정상리뷰
2022. 5. 9. 21:33
정상리뷰
2022. 5. 23. 11:19
정상리뷰
2022. 5. 26. 15:40
정상리뷰
2022. 6. 14. 9:19
정상리뷰
블로그링크접속실패
가게이름: 지아니스나폴리 가로수길점
블로그리스트 탐색중...
탐색끝
2022. 6. 27. 18:01
정상리뷰
2022. 2. 19. 0:

정상리뷰
2022. 6. 6. 10:42
정상리뷰
2022. 5. 19. 8:10
정상리뷰
2022. 5. 9. 13:40
정상리뷰
2022. 5. 28. 7:00
정상리뷰
2022. 5. 25. 23:20
광고리뷰확인
2022. 5. 9. 16:10
정상리뷰
2022. 5. 5. 9:57
정상리뷰
2022. 5. 9. 18:09
정상리뷰
2022. 2. 16. 17:51
정상리뷰
2022. 5. 6. 4:55
정상리뷰
2022. 4. 20. 6:37
정상리뷰
2022. 4. 20. 11:41
정상리뷰
2022. 4. 1. 16:19
정상리뷰
2022. 4. 4. 8:47
정상리뷰
2022. 4. 6. 23:05
정상리뷰
2022. 3. 12. 20:32
정상리뷰
2022. 3. 18. 17:10
정상리뷰
2022. 3. 14. 21:00
정상리뷰
2022. 4. 4. 21:26
정상리뷰
2022. 3. 8. 14:21
정상리뷰
2022. 3. 24. 16:00
정상리뷰
2022. 3. 22. 2:08
정상리뷰
2022. 2. 13. 1:33
정상리뷰
2022. 5. 10. 8:13
정상리뷰
2022. 2. 28. 12:00
정상리뷰
2022. 3. 28. 14:53
광고리뷰확인
2022. 2. 24. 0:49
정상리뷰
2022. 2. 23. 8:10
정상리뷰
2022. 2. 3. 1:40
정상리뷰
2022. 5. 24. 9:57
정상리뷰
2022. 2. 8. 23:00
정상리뷰
2022. 3. 17. 8:11
정상리뷰
2022. 1. 11. 23:55
정상리뷰
2021. 11. 22. 6:16
정상리뷰
2021. 9. 27. 22:21
정상리뷰
2021. 9. 10. 16:41
정상리뷰
2021. 10. 27. 16:26
정상리뷰
2021. 8. 22. 7:00
정상리뷰
2021. 10. 13. 14:54
정상리뷰
2021. 12. 2. 8:20
정상리뷰
2022. 5. 9. 16:38
정상리뷰
2021. 10. 18. 9:50
정상리뷰
2022. 5. 9.

2022. 4. 8. 7:46
정상리뷰
2022. 4. 29. 8:29
정상리뷰
2022. 4. 7. 8:00
정상리뷰
2022. 5. 11. 0:30
정상리뷰
2022. 4. 12. 7:32
정상리뷰
2022. 6. 19. 12:36
정상리뷰
블로그링크접속실패
가게이름: 효계
블로그리스트 탐색중...
탐색끝
2022. 6. 6. 7:30
정상리뷰
2022. 5. 16. 17:07
정상리뷰
2022. 4. 6. 11:34
정상리뷰
2022. 5. 28. 8:51
정상리뷰
22시간 전
정상리뷰
2022. 6. 23. 23:35
정상리뷰
2022. 6. 20. 10:22
정상리뷰
2022. 6. 18. 15:38
정상리뷰
2022. 2. 9. 8:20
정상리뷰
2022. 2. 26. 21:05
정상리뷰
2021. 12. 10. 8:02
정상리뷰
2022. 6. 22. 13:38
정상리뷰
2022. 6. 25. 17:32
정상리뷰
2020. 11. 30. 5:00
정상리뷰
2021. 3. 9. 18:23
정상리뷰
2022. 6. 12. 22:04
정상리뷰
2020. 8. 5. 7:10
정상리뷰
2020. 11. 29. 10:22
정상리뷰
2020. 9. 23. 7:10
정상리뷰
2022. 5. 27. 13:20
정상리뷰
2020. 9. 16. 10:38
정상리뷰
2020. 10. 12. 23:45
정상리뷰
2020. 9. 2. 5:00
정상리뷰
2022. 6. 12. 18:31
정상리뷰
2022. 6. 10. 11:05
정상리뷰
2022. 6. 3. 17:43
정상리뷰
2020. 10. 1. 22:18
정상리뷰
2022. 4. 18. 17:17
정상리뷰
2022. 3. 29. 12:43
정상리뷰
2022. 4. 24. 11:26
정상리뷰
2022. 3. 21. 9:50
정상리뷰
2022. 4. 15. 17:25
정상리뷰
2022. 4. 18. 18:15
정상리뷰
2022. 4. 27. 1:05
광고리뷰확인
2022. 3. 26. 13:34
정상리뷰
2022. 4. 

정상리뷰
2021. 12. 20. 9:30
정상리뷰
2022. 6. 26. 15:17
정상리뷰
2021. 12. 7. 16:36
정상리뷰
2021. 3. 16. 8:06
정상리뷰
2021. 10. 4. 17:04
정상리뷰
2020. 10. 24. 23:50
정상리뷰
2020. 2. 28. 3:26
정상리뷰
2021. 8. 2. 15:07
정상리뷰
2021. 8. 9. 22:42
정상리뷰
2021. 3. 14. 21:07
정상리뷰
2021. 7. 7. 16:52
정상리뷰
2022. 6. 21. 0:27
정상리뷰
2021. 2. 3. 23:39
정상리뷰
2021. 3. 18. 6:58
정상리뷰
2020. 5. 2. 15:11
정상리뷰
2022. 6. 12. 10:00
정상리뷰
2022. 3. 30. 8:00
정상리뷰
2022. 6. 16. 1:58
정상리뷰
2022. 4. 29. 16:01
정상리뷰
2022. 6. 15. 1:31
정상리뷰
2022. 6. 11. 21:49
정상리뷰
2022. 4. 20. 8:00
정상리뷰
2022. 5. 13. 0:20
정상리뷰
2022. 6. 10. 19:33
정상리뷰
2022. 6. 20. 15:53
정상리뷰
2022. 6. 12. 20:19
정상리뷰
2022. 5. 19. 10:20
정상리뷰
2022. 6. 7. 20:19
정상리뷰
2022. 4. 17. 20:41
정상리뷰
2022. 2. 14. 17:44
정상리뷰
2022. 5. 2. 22:27
정상리뷰
2022. 6. 2. 23:34
정상리뷰
2022. 5. 22. 14:58
정상리뷰
2022. 6. 1. 16:05
정상리뷰
2022. 5. 28. 14:08
정상리뷰
블로그링크접속실패
가게이름: 인터컨티넨탈 서울 코엑스 브래서리
블로그리스트 탐색중...
탐색끝
2022. 3. 17. 15:58
정상리뷰
2022. 6. 19. 18:50
정상리뷰
2022. 6. 22. 17:54
정상리뷰
2022. 6. 8. 15:10
정상리뷰
2022. 6. 4. 0:16
정상리뷰
20